In [2]:
import pandas as pd
import numpy as np
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from collections import Counter

In [3]:
max_words = 1000
random_state = 4

In [4]:
nltk.download('punkt_tab')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\esimb\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\esimb\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\esimb\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [9]:
df = pd.read_csv("../data/raw/Smart_Farming_Crop_Yield_2024.csv")
df


,Unnamed: 0,region,crop_type,soil_moisture_%,soil_pH,temperature_C,rainfall_mm,humidity_%,sunlight_hours,irrigation_type,...,yield_kg_per_hectare,timestamp,latitude,longitude,NDVI_index,crop_disease_status,region_le,croptype_le,cropdisease_le,irrigation_type_le
0,0,North India,Wheat,35.95,5.99,17.79,75.62,77.03,7.27,None,...,4408.07,2024-03-19,14.970941,82.997689,0.63,Mild,2,4,0,2
1,1,South USA,Soybean,19.74,7.24,30.18,89.91,61.13,5.67,Sprinkler,...,5389.98,2024-04-21,16.613022,70.869009,0.58,None,4,3,2,3
2,2,South USA,Wheat,29.32,7.16,27.37,265.43,68.87,8.23,Drip,...,2931.16,2024-02-28,19.503156,79.068206,0.80,Mild,4,4,0,0
3,3,Central USA,Maize,17.33,6.03,33.73,212.01,70.46,5.03,Sprinkler,...,4227.80,2024-05-14,31.071298,85.519998,0.44,None,0,1,2,3
4,4,Central USA,Cotton,19.37,5.92,33.86,269.09,55.73,7.93,None,...,4979.96,2024-04-13,16.568540,81.691720,0.84,Severe,0,0,3,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,495,Central USA,Rice,42.85,6.70,30.85,52.35,79.58,7.25,Manual,...,4251.40,2024-05-08,30.386623,76.147700,0.59,Mild,0,2,0,1
496,496,North India,Soybean,34.22,6.75,17.46,256.23,45.14,5.78,None,...,3708.54,2024-01-19,18.832748,75.736924,0.85,Severe,2,3,3,2
497,497,North India,Cotton,15.93,5.72,17.03,288.96,57.87,7.69,Drip,...,2604.41,2024-04-20,23.262016,81.992230,0.71,Mild,2,0,0,0
498,498,Central USA,Soybean,38.61,6.20,17.08,279.06,73.09,9.60,Drip,...,2586.36,2024-03-02,19.764989,84.426869,0.77,Severe,0,3,3,0


In [6]:
df.drop(columns=["farm_id", "sensor_id"], inplace=True) #clear the id columns
df.to_csv("../data/raw/Smart_Farming_Crop_Yield_2024.csv")
df

KeyError: "['farm_id', 'sensor_id'] not found in axis"

In [ ]:
def preprocess(text, stop_words, punctuation_marks, morph): #for the future
    tokens = word_tokenize(text.lower())
    preprocessed_text = []
    for token in tokens:
      if token not in punctuation_marks:
        lemma = morph.lemmatize(token)
        if lemma not in stop_words:
          preprocessed_text.append(lemma)
    return preprocessed_text


In [ ]:
punctuation_marks = ['!', ',', '(', ')', ':', '-', '?', '.', '..', '...', '/', '@', '1', '2', '3', '4', '5', '6', '7', '8', '9', '0', '"']
morph = WordNetLemmatizer() # for the future

In [ ]:
stop_words = stopwords.words("english")

In [10]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
df["fertilizer_type_le"] = le.fit_transform(df["fertilizer_type"])

df.head(8)

df.to_csv("../data/raw/2Smart_Farming_Crop_Yield_2024.csv")

In [ ]:
new_df.head()
new_df.to_csv("../data/preprocessed/Smart_Farming_Crop_Yield_2024.csv")

In [12]:
df.head() #i deleted all future features, to avoid the problems during the train

,Unnamed: 0,region,crop_type,soil_moisture_%,soil_pH,temperature_C,rainfall_mm,humidity_%,sunlight_hours,irrigation_type,...,timestamp,latitude,longitude,NDVI_index,crop_disease_status,region_le,croptype_le,cropdisease_le,irrigation_type_le,fertilizer_type_le
0,0,North India,Wheat,35.95,5.99,17.79,75.62,77.03,7.27,None,...,2024-03-19,14.970941,82.997689,0.63,Mild,2,4,0,2,2
1,1,South USA,Soybean,19.74,7.24,30.18,89.91,61.13,5.67,Sprinkler,...,2024-04-21,16.613022,70.869009,0.58,None,4,3,2,3,0
2,2,South USA,Wheat,29.32,7.16,27.37,265.43,68.87,8.23,Drip,...,2024-02-28,19.503156,79.068206,0.80,Mild,4,4,0,0,1
3,3,Central USA,Maize,17.33,6.03,33.73,212.01,70.46,5.03,Sprinkler,...,2024-05-14,31.071298,85.519998,0.44,None,0,1,2,3,2
4,4,Central USA,Cotton,19.37,5.92,33.86,269.09,55.73,7.93,None,...,2024-04-13,16.568540,81.691720,0.84,Severe,0,0,3,2,1


In [13]:
df["sowing_month"] = pd.to_datetime(df["sowing_date"]).dt.month
df["sowing_season"] = pd.to_datetime(df["sowing_date"]).dt.month % 12 // 3 + 1
df.head()


,Unnamed: 0,region,crop_type,soil_moisture_%,soil_pH,temperature_C,rainfall_mm,humidity_%,sunlight_hours,irrigation_type,...,longitude,NDVI_index,crop_disease_status,region_le,croptype_le,cropdisease_le,irrigation_type_le,fertilizer_type_le,sowing_month,sowing_season
0,0,North India,Wheat,35.95,5.99,17.79,75.62,77.03,7.27,None,...,82.997689,0.63,Mild,2,4,0,2,2,1,1
1,1,South USA,Soybean,19.74,7.24,30.18,89.91,61.13,5.67,Sprinkler,...,70.869009,0.58,None,4,3,2,3,0,2,1
2,2,South USA,Wheat,29.32,7.16,27.37,265.43,68.87,8.23,Drip,...,79.068206,0.80,Mild,4,4,0,0,1,2,1
3,3,Central USA,Maize,17.33,6.03,33.73,212.01,70.46,5.03,Sprinkler,...,85.519998,0.44,None,0,1,2,3,2,2,1
4,4,Central USA,Cotton,19.37,5.92,33.86,269.09,55.73,7.93,None,...,81.691720,0.84,Severe,0,0,3,2,1,2,1


In [26]:
df.drop(columns=["Unnamed: 0"], inplace=True)
df.head()

,soil_moisture_%,soil_pH,temperature_C,rainfall_mm,humidity_%,sunlight_hours,pesticide_usage_ml,yield_kg_per_hectare,latitude,longitude,region_le,croptype_le,cropdisease_le,irrigation_type_le,fertilizer_type_le,sowing_month,sowing_season
0,35.95,5.99,17.79,75.62,77.03,7.27,6.34,4408.07,14.970941,82.997689,2,4,0,2,2,1,1
1,19.74,7.24,30.18,89.91,61.13,5.67,9.60,5389.98,16.613022,70.869009,4,3,2,3,0,2,1
2,29.32,7.16,27.37,265.43,68.87,8.23,15.26,2931.16,19.503156,79.068206,4,4,0,0,1,2,1
3,17.33,6.03,33.73,212.01,70.46,5.03,25.80,4227.80,31.071298,85.519998,0,1,2,3,2,2,1
4,19.37,5.92,33.86,269.09,55.73,7.93,25.65,4979.96,16.568540,81.691720,0,0,3,2,1,2,1


In [27]:
df.isnull().sum()
df.describe()

,soil_moisture_%,soil_pH,temperature_C,rainfall_mm,humidity_%,sunlight_hours,pesticide_usage_ml,yield_kg_per_hectare,latitude,longitude,region_le,croptype_le,cropdisease_le,irrigation_type_le,fertilizer_type_le,sowing_month,sowing_season
count,500.000000,500.000000,500.000000,500.000000,500.000000,500.00000,500.000000,500.00000,500.000000,500.000000,500.000000,500.000000,500.000000,500.00000,500.000000,500.000000,500.000000
mean,26.750140,6.523980,24.675740,181.685740,65.194460,7.03014,26.586980,4032.92694,22.442473,80.392248,1.908000,1.934000,1.542000,1.56200,0.998000,2.062000,1.374000
std,10.150053,0.585558,5.348899,72.293091,14.642849,1.69167,13.202429,1174.43304,7.283492,5.910664,1.419707,1.424677,1.132608,1.08467,0.816903,0.826755,0.484348
min,10.160000,5.510000,15.000000,50.170000,40.230000,4.01000,5.050000,2023.56000,10.004243,70.020021,0.000000,0.000000,0.000000,0.00000,0.000000,1.000000,1.000000
25%,17.890000,6.030000,20.295000,119.217500,51.865000,5.66750,14.945000,2994.82000,16.263202,75.374713,1.000000,1.000000,0.750000,1.00000,0.000000,1.000000,1.000000
50%,25.855000,6.530000,24.655000,191.545000,65.685000,6.99500,25.980000,4071.69000,21.981743,80.650284,2.000000,2.000000,2.000000,2.00000,1.000000,2.000000,1.000000
75%,36.022500,7.040000,29.090000,239.035000,77.995000,8.47000,38.005000,5062.11000,28.528948,85.654629,3.000000,3.000000,3.000000,2.00000,2.000000,3.000000,2.000000
max,44.980000,7.500000,34.840000,298.960000,90.000000,10.00000,49.940000,5998.29000,34.981531,89.991901,4.000000,4.000000,3.000000,3.00000,2.000000,3.000000,2.000000


In [28]:
y = df["yield_kg_per_hectare"] #Our target to predict 
X = df.drop(columns=["yield_kg_per_hectare"]) #Features
y.head()
X.head()

,soil_moisture_%,soil_pH,temperature_C,rainfall_mm,humidity_%,sunlight_hours,pesticide_usage_ml,latitude,longitude,region_le,croptype_le,cropdisease_le,irrigation_type_le,fertilizer_type_le,sowing_month,sowing_season
0,35.95,5.99,17.79,75.62,77.03,7.27,6.34,14.970941,82.997689,2,4,0,2,2,1,1
1,19.74,7.24,30.18,89.91,61.13,5.67,9.60,16.613022,70.869009,4,3,2,3,0,2,1
2,29.32,7.16,27.37,265.43,68.87,8.23,15.26,19.503156,79.068206,4,4,0,0,1,2,1
3,17.33,6.03,33.73,212.01,70.46,5.03,25.80,31.071298,85.519998,0,1,2,3,2,2,1
4,19.37,5.92,33.86,269.09,55.73,7.93,25.65,16.568540,81.691720,0,0,3,2,1,2,1


In [29]:
y.head()

0    4408.07
1    5389.98
2    2931.16
3    4227.80
4    4979.96
Name: yield_kg_per_hectare, dtype: float64

In [30]:
X.head()

,soil_moisture_%,soil_pH,temperature_C,rainfall_mm,humidity_%,sunlight_hours,pesticide_usage_ml,latitude,longitude,region_le,croptype_le,cropdisease_le,irrigation_type_le,fertilizer_type_le,sowing_month,sowing_season
0,35.95,5.99,17.79,75.62,77.03,7.27,6.34,14.970941,82.997689,2,4,0,2,2,1,1
1,19.74,7.24,30.18,89.91,61.13,5.67,9.60,16.613022,70.869009,4,3,2,3,0,2,1
2,29.32,7.16,27.37,265.43,68.87,8.23,15.26,19.503156,79.068206,4,4,0,0,1,2,1
3,17.33,6.03,33.73,212.01,70.46,5.03,25.80,31.071298,85.519998,0,1,2,3,2,2,1
4,19.37,5.92,33.86,269.09,55.73,7.93,25.65,16.568540,81.691720,0,0,3,2,1,2,1


In [31]:
X.to_csv("../data/preprocessed/trainig_data.csv")
y.to_csv("../data/preprocessed/target_data.csv")

In [8]:
data = pd.read_csv("../data/raw/Smart_Farming_Crop_Yeild_2024.csv")
data.head()

FileNotFoundError: [Errno 2] No such file or directory: '../data/raw/Smart_Farming_Crop_Yeild_2024.csv'